# Crawl match instances and performance instances

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time

driver = webdriver.Chrome()
#driver = webdriver.Safari()
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/Fixtures/Italy-Serie-A-2016-2017"
driver.get(start_url)

In [28]:
# Find the link of each match
match_table = {}

prev_month_button = driver.find_element_by_xpath("//*[@id=\"date-controller\"]/a[1]")
prev_button_status = prev_month_button.get_attribute("title")
previous_month = ""
current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text

while (prev_button_status == "View previous month"):
    while (current_month == previous_month):
        prev_month_button.click()
        time.sleep(1)
        current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text
        prev_button_status = prev_month_button.get_attribute("title")

    while True:
        try:
            # match_table = {}
            trs = driver.find_elements_by_xpath("//*[@id=\"tournament-fixture\"]/tbody/tr")

            for tr in trs:
                tds = tr.find_elements_by_xpath("td")
                if(len(tds)):
                    match_id = tr.get_attribute("data-id")
                    match_table[match_id] = {}
                for i, td in enumerate(tds):
                    if(i == 4):
                        match_table[match_id]["match_link"] = td.find_element_by_tag_name("a").get_attribute("href")
            break
        except:
            print("wait")
            time.sleep(0.5)

    previous_month = current_month

In [30]:
# Crawl data of each match details
print("Total number of matches:{}.".format(len(match_table)))
cnt = 1
subdriver = webdriver.Chrome()
for match_id, match_details in match_table.items():
    match_link = match_details["match_link"]
    # Change into the match page
    print(">>> {}: Crawling {}...".format(cnt, match_id))
    cnt += 1
    
    subdriver.get(match_link)
    time.sleep(1)
    web_match_trs = subdriver.find_elements_by_xpath("//*[@id=\"match-header\"]/table/tbody/tr")

    # Crawl basic match information
    web_match_tr = web_match_trs[0]
    web_match_tds = web_match_tr.find_elements_by_tag_name("td")
    home_team_id = int(re.findall(r'\d+', web_match_tds[0].find_element_by_tag_name("a").get_attribute("href"))[0])
    away_team_id = int(re.findall(r'\d+', web_match_tds[2].find_element_by_tag_name("a").get_attribute("href"))[0])

    web_match_tr = web_match_trs[1]
    web_match_tds = web_match_tr.find_elements_by_tag_name("td")
    web_match_divs = web_match_tds[1].find_elements_by_tag_name("div")
    match_datetime = web_match_divs[2].find_element_by_xpath("dl/dd[2]").text + ", " + web_match_divs[2].find_element_by_xpath("dl/dd[1]").text
    half_time_result = web_match_divs[1].find_element_by_xpath("dl/dd[1]").text
    result = web_match_divs[1].find_element_by_xpath("dl/dd[2]").text
    
    match_table[match_id]["match_datetime"] = match_datetime
    match_table[match_id]["home_team_id"] = home_team_id
    match_table[match_id]["away_team_id"] = away_team_id
    match_table[match_id]["half_time_result"] = half_time_result
    match_table[match_id]["result"] = result
print(">>> finished.")
driver.close()
subdriver.close()

Total number of matches:380.
>>> 1: Crawling 1115280...
>>> 2: Crawling 1115216...
>>> 3: Crawling 1115389...
>>> 4: Crawling 1115500...
>>> 5: Crawling 1115205...
>>> 6: Crawling 1115298...
>>> 7: Crawling 1115445...
>>> 8: Crawling 1115170...
>>> 9: Crawling 1115508...
>>> 10: Crawling 1115338...
>>> 11: Crawling 1115277...
>>> 12: Crawling 1115505...
>>> 13: Crawling 1115251...
>>> 14: Crawling 1115368...
>>> 15: Crawling 1115525...
>>> 16: Crawling 1115272...
>>> 17: Crawling 1115348...
>>> 18: Crawling 1115323...
>>> 19: Crawling 1115401...
>>> 20: Crawling 1115387...
>>> 21: Crawling 1115187...
>>> 22: Crawling 1115380...
>>> 23: Crawling 1115414...
>>> 24: Crawling 1115188...
>>> 25: Crawling 1115275...
>>> 26: Crawling 1115201...
>>> 27: Crawling 1115150...
>>> 28: Crawling 1115303...
>>> 29: Crawling 1115247...
>>> 30: Crawling 1115185...
>>> 31: Crawling 1115200...
>>> 32: Crawling 1115154...
>>> 33: Crawling 1115190...
>>> 34: Crawling 1115373...
>>> 35: Crawling 1115420...


>>> 287: Crawling 1115263...
>>> 288: Crawling 1115480...
>>> 289: Crawling 1115347...
>>> 290: Crawling 1115492...
>>> 291: Crawling 1115455...
>>> 292: Crawling 1115412...
>>> 293: Crawling 1115393...
>>> 294: Crawling 1115294...
>>> 295: Crawling 1115240...
>>> 296: Crawling 1115197...
>>> 297: Crawling 1115399...
>>> 298: Crawling 1115470...
>>> 299: Crawling 1115331...
>>> 300: Crawling 1115495...
>>> 301: Crawling 1115152...
>>> 302: Crawling 1115227...
>>> 303: Crawling 1115155...
>>> 304: Crawling 1115382...
>>> 305: Crawling 1115181...
>>> 306: Crawling 1115194...
>>> 307: Crawling 1115210...
>>> 308: Crawling 1115419...
>>> 309: Crawling 1115488...
>>> 310: Crawling 1115169...
>>> 311: Crawling 1115427...
>>> 312: Crawling 1115288...
>>> 313: Crawling 1115186...
>>> 314: Crawling 1115522...
>>> 315: Crawling 1115365...
>>> 316: Crawling 1115411...
>>> 317: Crawling 1115375...
>>> 318: Crawling 1115295...
>>> 319: Crawling 1115245...
>>> 320: Crawling 1115466...
>>> 321: Crawl

In [33]:
import json

with open('match_instances_table.json', 'w') as fp:
    json.dump(match_table, fp, indent=4)

### Example for reading JSON as Dataframe

In [37]:
import pandas as pd
tmp = pd.read_json('match_instances_table.json', orient='index')

# Crawl team instances

In [76]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time

driver = webdriver.Chrome()
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/TeamStatistics/Italy-Serie-A-2016-2017"
driver.get(start_url)

In [77]:
team_table = {}

trs = driver.find_elements_by_xpath("//*[@id=\"top-team-stats-summary-content\"]/tr")

for tr in trs:
    team_link = tr.find_element_by_class_name("team-link").get_attribute("href")
    team_id = int(re.findall(r'\d+',team_link)[0])
    team_table[team_id] = {}
    team_table[team_id]["team_link"] = team_link

In [78]:
# Crawl data of each team
print("Total number of teams:{}.".format(len(team_table)))
cnt = 1
subdriver = webdriver.Chrome()
for team_id, team_details in team_table.items():
    team_link = team_details['team_link']
    # Change into the team page
    print(">>> {}: Crawling {}...".format(cnt, team_id))
    
    while True:
        try:
            subdriver.get(team_link)
            time.sleep(1)

            web_team_img_link = subdriver.find_element_by_xpath("//*[@id=\"layout-content-wrapper\"]/div[2]/div[1]/h2/img").get_attribute("src")
            web_team_name = subdriver.find_element_by_xpath("//*[@id=\"layout-content-wrapper\"]/div[2]/div[1]/h2/span").text
            break
        except:
            time.sleep(1)
    
    team_table[team_id]["team_img_link"] = web_team_img_link
    team_table[team_id]["team_name"] = web_team_name
    
    cnt += 1
    
print(">>> finished.")
driver.close()
subdriver.close()

Total number of teams:20.
>>> 1: Crawling 272...
>>> 2: Crawling 267...
>>> 3: Crawling 779...
>>> 4: Crawling 278...
>>> 5: Crawling 71...
>>> 6: Crawling 72...
>>> 7: Crawling 73...
>>> 8: Crawling 277...
>>> 9: Crawling 75...
>>> 10: Crawling 300...
>>> 11: Crawling 77...
>>> 12: Crawling 2889...
>>> 13: Crawling 271...
>>> 14: Crawling 80...
>>> 15: Crawling 276...
>>> 16: Crawling 78...
>>> 17: Crawling 86...
>>> 18: Crawling 87...
>>> 19: Crawling 84...
>>> 20: Crawling 1285...


In [79]:
import json

with open('team_instances_table.json', 'w') as fp:
    json.dump(team_table, fp, indent=4)

### Example for reading JSON as Dataframe

In [68]:
import pandas as pd
tmp = pd.read_json('team_instances_table.json', orient='index')

# Crawl player instances

In [101]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time

driver = webdriver.Chrome()
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/PlayerStatistics/Italy-Serie-A-2016-2017"
driver.get(start_url)

ALL_PLAYER = True

In [102]:
player_table = {}

if ALL_PLAYER:
    all_players_button = driver.find_element_by_xpath("//*[@id=\"apps\"]/dd[2]/a")
    all_players_button.click()

time.sleep(1)

In [103]:
next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
next_button_status = next_button.get_attribute("class")

player_cnt = 0
player_id_list = []
cnt = 1

while next_button_status != "option  disabled ":
    print(">>> Crawling page {}...".format(cnt))
    while True:
        try:
            player_statistic_tbody = driver.find_element_by_xpath("//*[@id=\"player-table-statistics-body\"]")
            player_trs = player_statistic_tbody.find_elements_by_xpath("tr")
            for tr in player_trs:
                pn = tr.find_element_by_class_name("pn")
                player_link = pn.find_element_by_class_name("player-link").get_attribute("href")
                player_id = int(re.findall(r'\d+', player_link)[0])

                player_table[player_id] = {}
                player_table[player_id]["player_link"] = player_link

                player_id_list.append(player_id)
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
    
    while True:
        try:
            next_button.click()
            break
        except:
            print(">>> error...waiting next click...")
            time.sleep(1)
            next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
    cnt += 1
    next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
    next_button_status = next_button.get_attribute("class")

print(">>> finished.")

>>> Crawling page 1...
>>> Crawling page 2...
>>> Crawling page 3...
>>> Crawling page 4...
>>> Crawling page 5...
>>> Crawling page 6...
>>> Crawling page 7...
>>> Crawling page 8...
>>> Crawling page 9...
>>> Crawling page 10...
>>> Crawling page 11...
>>> Crawling page 12...
>>> Crawling page 13...
>>> Crawling page 14...
>>> Crawling page 15...
>>> Crawling page 16...
>>> Crawling page 17...
>>> Crawling page 18...
>>> Crawling page 19...
>>> Crawling page 20...
>>> Crawling page 21...
>>> Crawling page 22...
>>> Crawling page 23...
>>> Crawling page 24...
>>> Crawling page 25...
>>> Crawling page 26...
>>> Crawling page 27...
>>> Crawling page 28...
>>> Crawling page 29...
>>> Crawling page 30...
>>> Crawling page 31...
>>> Crawling page 32...
>>> Crawling page 33...
>>> Crawling page 34...
>>> Crawling page 35...
>>> Crawling page 36...
>>> Crawling page 37...
>>> Crawling page 38...
>>> Crawling page 39...
>>> Crawling page 40...
>>> Crawling page 41...
>>> Crawling page 42...
>

In [105]:
# Crawl data of each player
print("Total number of players:{}.".format(len(player_table)))
cnt = 1
subdriver = webdriver.Chrome()
for player_id, player_details in player_table.items():
    player_link = player_details['player_link']
    # Change into the player page
    print(">>> {}: Crawling {}...".format(cnt, player_id))

    while True:
        try:
            subdriver.get(player_link)
            time.sleep(1)

            web_player_name = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl[1]/dd").text
            web_player_img_link = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/img").get_attribute("src")
            web_player_birthday = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[1]/dd/i").text        
            web_player_height = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[2]/dd").text)[0])
            web_player_weight = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[3]/dd").text)[0])
            team_id = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl[2]/dd/a").get_attribute("href"))[0])
            no_dls = subdriver.find_elements_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl")
            web_player_position_items = no_dls[-1].find_elements_by_xpath("dd/ul/li")
            web_player_position = [item.text for item in web_player_position_items]
            
            player_table[player_id]["player_name"] = web_player_name   
            player_table[player_id]["player_img_link"] = web_player_img_link
            player_table[player_id]["player_birthday"] = web_player_birthday
            player_table[player_id]["player_height"] = web_player_height
            player_table[player_id]["player_weight"] = web_player_weight
            player_table[player_id]["team_id"] = team_id
            player_table[player_id]["player_position"] = web_player_position
            
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
        
    cnt += 1

print(">>> finished")
driver.close()
subdriver.close()   

Total number of players:558.
>>> 1: Crawling 141312...
>>> 2: Crawling 6145...
>>> 3: Crawling 68613...
>>> 4: Crawling 40966...
>>> 5: Crawling 131079...
>>> 6: Crawling 141320...
>>> 7: Crawling 249865...
>>> 8: Crawling 98317...
>>> 9: Crawling 23567...
>>> 10: Crawling 118801...
>>> 11: Crawling 22546...
>>> 12: Crawling 70675...
>>> 13: Crawling 90132...
>>> 14: Crawling 6166...
>>> 15: Crawling 82967...
>>> 16: Crawling 106522...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 17: Crawling 90287...
>>> 18: Crawling 24605...
>>> error...waiting next trial...
>>> 19: Crawling 135198...
>>> 20: Crawling 78880...
>>> 21: Crawling 13348...
>>> 22: Crawling 82438...
>>> 23: Crawling 67261...
>>> 24: Crawling 84008...
>>> 25: Crawling 53975...
>>> 26: Crawling 33834...
>>> 27: Crawling 71687...
>>> 28: Crawling 78892...
>>> 29: Crawling 15407...
>>> 30: Crawling 

>>> 213: Crawling 300438...
>>> 214: Crawling 41367...
>>> 215: Crawling 87449...
>>> 216: Crawling 14063...
>>> 217: Crawling 135581...
>>> 218: Crawling 14751...
>>> 219: Crawling 135584...
>>> 220: Crawling 303856...
>>> 221: Crawling 322971...
>>> 222: Crawling 141736...
>>> 223: Crawling 28963...
>>> 224: Crawling 76202...
>>> 225: Crawling 105562...
>>> 226: Crawling 14066...
>>> 227: Crawling 141725...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 228: Crawling 109410...
>>> 229: Crawling 106931...
>>> error...waiting n

>>> 432: Crawling 128790...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 433: Crawling 120601...
>>> 434: Crawling 90906...
>>> error...waiting next trial...
>>> 435: Crawling 27421...
>>> 436: Crawling 10015...
>>> 437: Crawling 71458...
>>> 438: Crawling 106275...
>>> 439: Crawling 98126...
>>> 440: Crawling 323240...
>>> 441: Crawling 302893...
>>> 442: Crawling 14126...
>>> 443: Crawling 14127...
>>> 444: Crawling 23043...
>>> 445: Crawling 22321...
>>> 446: Crawling 234291...
>>> 447: Crawling 35640...
>>> 448: Crawling 108345...
>>> 449: Crawling 79674...
>>> 450: Crawling 122683...
>>> 451: Crawling 2877...
>>> 452: Crawling 104257...
>>> 453: Crawling 90946...
>>> 454: Crawling 44868...
>>> 455: Crawling 302918...
>>> 456: Crawling 10209...
>>> 457: Crawling 14152...
>>> 458: Crawling 91978...
>>> 459: Crawling 103242...
>>> 460: Crawling 14157...
>>> 461: Crawling 59214...
>>> 462: Crawling 109197...
>>> 463: Crawling 109393...
>>> 464: Crawling 1401

In [108]:
import json

with open('player_instances_table.json', 'w') as fp:
    json.dump(player_table, fp, indent=4)

# Crawl performance instances

In [109]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time

driver = webdriver.Chrome()
#driver = webdriver.Safari()
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/Fixtures/Italy-Serie-A-2016-2017"
driver.get(start_url)

In [110]:
# Find the link of each match
match_table = {}

prev_month_button = driver.find_element_by_xpath("//*[@id=\"date-controller\"]/a[1]")
prev_button_status = prev_month_button.get_attribute("title")
previous_month = ""
current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text

while (prev_button_status == "View previous month"):
    while (current_month == previous_month):
        prev_month_button.click()
        time.sleep(1)
        current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text
        prev_button_status = prev_month_button.get_attribute("title")

    while True:
        try:
            # match_table = {}
            trs = driver.find_elements_by_xpath("//*[@id=\"tournament-fixture\"]/tbody/tr")

            for tr in trs:
                tds = tr.find_elements_by_xpath("td")
                if(len(tds)):
                    match_id = tr.get_attribute("data-id")
                    match_table[match_id] = {}
                for i, td in enumerate(tds):
                    if(i == 4):
                        match_table[match_id]["match_link"] = td.find_element_by_tag_name("a").get_attribute("href")
            break
        except:
            print("wait")
            time.sleep(0.5)

    previous_month = current_month

In [111]:
# Crawl data of each match performance
performance_table = {}

print("Total number of matches:{}.".format(len(match_table)))
cnt = 1
subdriver = webdriver.Chrome()
for match_id, match_details in match_table.items():
    performance_table[match_id] = {}
    match_link = match_details["match_link"]
    # Change into the match page
    print(">>> {}: Crawling {}...".format(cnt, match_id))
    cnt += 1
    
    while True:
        try:
            subdriver.get(match_link)
            time.sleep(1)
            
            # Record the match time first
            match_time = subdriver.find_element_by_xpath("//*[@id=\"match-centre-timeline\"]/div[1]/div[2]/div[2]/div[2]/div[2]/span").get_attribute("textContent")
            match_time = sum([int(digit) for digit in match_time.split("+")])
            
            # Change into player statistic page
            subdriver.get(subdriver.find_element_by_xpath("//*[@id=\"sub-sub-navigation\"]/ul/li[2]/a").get_attribute("href"))
            time.sleep(1)
            
            # Click through all player statistic tables such that they can be loaded
            home_offensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[2]/a")
            home_offensive_button.click()
            time.sleep(2)
            away_offensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[2]/a")
            away_offensive_button.click()
            time.sleep(2)

            # Crawl match performance
            #home_offensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-offensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_offensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-offensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #away_offensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-offensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_offensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-offensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")

            total_offensive_trs = home_offensive_trs + away_offensive_trs

            for tr in total_offensive_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])
                performance_table[match_id][player_id] = {}

                #total_shots = tr.find_element_by_class_name("ShotsTotal ").text
                total_shots = tr.find_element_by_xpath("td[4]").get_attribute("textContent")
                #ot_shots = tr.find_element_by_class_name("ShotOnTarget ").text
                ot_shots = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #key_passes = tr.find_element_by_class_name("KeyPassTotal ").text
                key_passes = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #dribbles = tr.find_element_by_class_name("DribbleWon ").text
                dribbles = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #foul_given = tr.find_element_by_class_name("FoulGiven ").text
                foul_given = tr.find_element_by_xpath("td[8]").get_attribute("textContent")
                #offsides = tr.find_element_by_class_name("OffsideGiven ").text
                offsides = tr.find_element_by_xpath("td[9]").get_attribute("textContent")
                #disp = tr.find_element_by_class_name("Dispossessed ").text
                disp = tr.find_element_by_xpath("td[10]").get_attribute("textContent")
                #uns_touches = tr.find_element_by_class_name("Turnover ").text
                uns_touches = tr.find_element_by_xpath("td[11]").get_attribute("textContent")

                rating = tr.find_element_by_xpath("td[12]").get_attribute("textContent")

                key_events_icons = tr.find_elements_by_class_name("incident-icon")
                key_events = ["{},{},{}".format(icon.get_attribute("data-type"), icon.get_attribute("data-minute"), icon.get_attribute("data-second")) for icon in key_events_icons]

                minutes_played = match_time

                for event in key_events:
                    event_type = event.split(",")[0]
                    event_minute = event.split(",")[1]
                    if (event_type == "18"):
                        minutes_played = int(event_minute)
                        break
                    elif (event_type == "19"):
                        minutes_played = match_time - int(event_minute)
                        break
                
                performance_table[match_id][player_id]["total_shots"] = re.sub('\t', '', total_shots)
                performance_table[match_id][player_id]["ot_shots"] = re.sub('\t', '', ot_shots)
                performance_table[match_id][player_id]["key_passes"] = re.sub('\t', '', key_passes)
                performance_table[match_id][player_id]["dribbles"] = re.sub('\t', '', dribbles)
                performance_table[match_id][player_id]["foul_given"] = re.sub('\t', '', foul_given)
                performance_table[match_id][player_id]["offsides"] = re.sub('\t', '', offsides)
                performance_table[match_id][player_id]["disp"] = re.sub('\t', '', disp)
                performance_table[match_id][player_id]["uns_touches"] = re.sub('\t', '', uns_touches)
                performance_table[match_id][player_id]["rating"] = rating
                performance_table[match_id][player_id]["key_events"] = key_events
                performance_table[match_id][player_id]["minutes_played"] = minutes_played

            home_defensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[3]/a")
            home_defensive_button.click()
            time.sleep(2)
            away_defensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[3]/a")
            away_defensive_button.click()
            time.sleep(2)

            #away_defensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-defensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_defensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-defensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #home_defensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-defensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_defensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-defensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")

            total_defensive_trs = home_defensive_trs + away_defensive_trs

            for tr in total_defensive_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])

                #total_tackles = tr.find_element_by_class_name("TackleWonTotal ").text
                total_tackles = tr.find_element_by_xpath("td[4]").get_attribute("textContent")
                #interception = tr.find_element_by_class_name("InterceptionAll ").text
                interception = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #clearances = tr.find_element_by_class_name("ClearanceTotal ").text
                clearances = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #blocked_shots = tr.find_element_by_class_name("ShotBlocked ").text
                blocked_shots = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #fouls_committed = tr.find_element_by_class_name("FoulCommitted ").text
                fouls_committed = tr.find_element_by_xpath("td[8]").get_attribute("textContent")

                performance_table[match_id][player_id]["total_tackles"] = re.sub('\t', '', total_tackles)
                performance_table[match_id][player_id]["interception"] = re.sub('\t', '', interception)
                performance_table[match_id][player_id]["clearances"] = re.sub('\t', '', clearances)
                performance_table[match_id][player_id]["blocked_shots"] = re.sub('\t', '', blocked_shots)
                performance_table[match_id][player_id]["fouls_committed"] = re.sub('\t', '', fouls_committed)

            home_passing_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[4]/a")
            home_passing_button.click()
            time.sleep(2)
            away_passing_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[4]/a")
            away_passing_button.click()
            time.sleep(2)

            #home_passing_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-passing\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_passing_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-passing\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #away_passing_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-passing\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_passing_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-passing\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            total_passing_trs = home_passing_trs + away_passing_trs

            for tr in total_passing_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])

                #total_passes = tr.find_element_by_class_name("TotalPasses ").text
                total_passes = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #pass_acc = tr.find_element_by_class_name("PassSuccessInMatch ").text
                pass_acc = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #crosses = tr.find_element_by_class_name("PassCrossTotal ").text
                crosses = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #acc_crosses = tr.find_element_by_class_name("PassCrossAccurate ").text
                acc_crosses = tr.find_element_by_xpath("td[8]").get_attribute("textContent")
                #total_long_balls = tr.find_element_by_class_name("PassLongBallTotal ").text
                total_long_balls = tr.find_element_by_xpath("td[9]").get_attribute("textContent")
                #acc_long_balls = tr.find_element_by_class_name("PassLongBallAccurate ").text
                acc_long_balls = tr.find_element_by_xpath("td[10]").get_attribute("textContent")
                #total_through_balls = tr.find_element_by_class_name("PassThroughBallTotal ").text
                total_through_balls = tr.find_element_by_xpath("td[11]").get_attribute("textContent")
                #acc_through_balls = tr.find_element_by_class_name("PassThroughBallAccurate ").text
                acc_through_balls = tr.find_element_by_xpath("td[12]").get_attribute("textContent")

                performance_table[match_id][player_id]["total_passes"] = re.sub('\t', '', total_passes)
                performance_table[match_id][player_id]["pass_acc"] = re.sub('\t', '', pass_acc)
                performance_table[match_id][player_id]["crosses"] = re.sub('\t', '', crosses)
                performance_table[match_id][player_id]["acc_crosses"] = re.sub('\t', '', acc_crosses)
                performance_table[match_id][player_id]["total_long_balls"] = re.sub('\t', '', total_long_balls)
                performance_table[match_id][player_id]["acc_long_balls"] = re.sub('\t', '', acc_long_balls)
                performance_table[match_id][player_id]["total_through_balls"] = re.sub('\t', '', total_through_balls)
                performance_table[match_id][player_id]["acc_through_balls"] = re.sub('\t', '', acc_through_balls)
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
    
print(">>> finished.")
driver.close()
subdriver.close()

Total number of matches:380.
>>> 1: Crawling 1115243...
>>> 2: Crawling 1115349...
>>> 3: Crawling 1115402...
>>> 4: Crawling 1115165...
>>> 5: Crawling 1115383...
>>> 6: Crawling 1115479...
>>> 7: Crawling 1115192...
>>> 8: Crawling 1115181...
>>> 9: Crawling 1115159...
>>> 10: Crawling 1115498...
>>> 11: Crawling 1115481...
>>> 12: Crawling 1115261...
>>> 13: Crawling 1115229...
>>> 14: Crawling 1115356...
>>> 15: Crawling 1115361...
>>> 16: Crawling 1115516...
>>> 17: Crawling 1115528...
>>> 18: Crawling 1115503...
>>> 19: Crawling 1115330...
>>> 20: Crawling 1115224...
>>> 21: Crawling 1115497...
>>> 22: Crawling 1115419...
>>> 23: Crawling 1115494...
>>> 24: Crawling 1115365...
>>> 25: Crawling 1115312...
>>> 26: Crawling 1115379...
>>> 27: Crawling 1115510...
>>> 28: Crawling 1115339...
>>> 29: Crawling 1115185...
>>> 30: Crawling 1115269...
>>> 31: Crawling 1115177...
>>> 32: Crawling 1115488...
>>> 33: Crawling 1115258...
>>> 34: Crawling 1115487...
>>> 35: Crawling 1115489...


>>> 284: Crawling 1115268...
>>> 285: Crawling 1115414...
>>> 286: Crawling 1115441...
>>> 287: Crawling 1115391...
>>> 288: Crawling 1115475...
>>> 289: Crawling 1115313...
>>> 290: Crawling 1115288...
>>> 291: Crawling 1115279...
>>> 292: Crawling 1115153...
>>> 293: Crawling 1115248...
>>> 294: Crawling 1115355...
>>> 295: Crawling 1115360...
>>> 296: Crawling 1115496...
>>> 297: Crawling 1115285...
>>> 298: Crawling 1115523...
>>> 299: Crawling 1115315...
>>> 300: Crawling 1115524...
>>> 301: Crawling 1115184...
>>> 302: Crawling 1115375...
>>> 303: Crawling 1115444...
>>> 304: Crawling 1115509...
>>> 305: Crawling 1115381...
>>> 306: Crawling 1115380...
>>> 307: Crawling 1115382...
>>> 308: Crawling 1115150...
>>> 309: Crawling 1115373...
>>> 310: Crawling 1115272...
>>> 311: Crawling 1115464...
>>> 312: Crawling 1115186...
>>> 313: Crawling 1115411...
>>> 314: Crawling 1115175...
>>> 315: Crawling 1115500...
>>> 316: Crawling 1115162...
>>> 317: Crawling 1115325...
>>> 318: Crawl

In [117]:
import json

with open('performance_instances_table.json', 'w') as fp:
    json.dump(performance_table, fp, indent=4)